In [ ]:
%python
import subprocess

# Packages to install
packages_to_install = ['kafka-python', 'ntscraper']

# Use subprocess to run the pip install commands
for package in packages_to_install:
    subprocess.call(['pip', 'install', package])


In [1]:
%python
import json
from kafka import KafkaProducer
from ntscraper import Nitter

# Define the Kafka Producer configuration
KAFKA_BOOTSTRAP_SERVERS = ['kafka-broker:29092']
KAFKA_TOPIC_NAME = 'sentiment_analysis'
KAFKA_PRODUCER_CONFIG = {
    'bootstrap_servers': KAFKA_BOOTSTRAP_SERVERS
}

terms = ["genocide", "gaza", "world"]
continents = ["Africa", "Asia", "Europe", "North America", "South America"]

def get_twitter_data(terms, continents):
    Twitter_data_list = []
    scraper = Nitter(0)

    for term in terms:
        for country in continents:
            tweets = scraper.get_tweets(term, mode='term', language='en', number=100, near=country)
            
            # Print information about the tweets variable
            # print(f"Term: {term}, Country: {country}, Tweets: {tweets}")
            
            for x in tweets['tweets']:
                    data = {
                        'text': x['text'],
                        'date': x['date'],
                        'likes': x['stats']['likes'],
                        'is_retweet': x['is-retweet'],
                        'retweets': x['stats']['retweets'],
                        'country': country  # Add the country name to the data
                    }
                    Twitter_data_list.append(data)

    return Twitter_data_list

def main(terms):
    try:
        producer = KafkaProducer(**KAFKA_PRODUCER_CONFIG)

        twitter_data = get_twitter_data(terms, continents)
        for tweet in twitter_data:
            json_data = json.dumps(tweet)
            print(json_data)
            producer.send(KAFKA_TOPIC_NAME, json_data.encode())

    except Exception as e:
        print(f'Error: {e}')

main(terms)


In [2]:

%python
